In [1]:
# 04_analyze_graph.ipynb
# -------------------------------------------------------------
# Purpose:
# Optional QA before or alongside Neo4j.
# Checks enriched data to confirm injected fraud patterns
# (shared phones, shared IPs, and overlaps).
# Exports Top 5 suspicious phone+IP pairs to assets as CSV + Markdown.
# -------------------------------------------------------------

import pandas as pd
from pathlib import Path

# ---- 1. Load enriched data
df = pd.read_csv("../data/processed/enriched_transactions.csv")

# ---- 2. Helper to clean columns
def _clean_series(s):
    s = s.astype(str).str.strip()
    s = s.replace({"nan": None, "None": None, "": None})
    return s

df["sender_phone"]   = _clean_series(df["sender_phone"])
df["receiver_phone"] = _clean_series(df["receiver_phone"])
df["sender_ip"]      = _clean_series(df["sender_ip"])
df["receiver_ip"]    = _clean_series(df["receiver_ip"])

# ---- 3. Build user→phone and user→IP maps
phones = pd.concat([
    df[["nameOrig", "sender_phone"]].rename(columns={"nameOrig": "user_id", "sender_phone": "phone"}),
    df[["nameDest", "receiver_phone"]].rename(columns={"nameDest": "user_id", "receiver_phone": "phone"})
], ignore_index=True)
phones = phones.dropna(subset=["phone"]).drop_duplicates()

ips = pd.concat([
    df[["nameOrig", "sender_ip"]].rename(columns={"nameOrig": "user_id", "sender_ip": "ip"}),
    df[["nameDest", "receiver_ip"]].rename(columns={"nameDest": "user_id", "receiver_ip": "ip"})
], ignore_index=True)
ips = ips.dropna(subset=["ip"]).drop_duplicates()

# ---- 4. Quick sanity counts
shared_phones_n = (phones.groupby("phone")["user_id"].nunique() > 1).sum()
shared_ips_n    = (ips.groupby("ip")["user_id"].nunique() > 1).sum()

print(f"Phones shared by >1 user: {shared_phones_n}")
print(f"IPs shared by >1 user: {shared_ips_n}")

# ---- 5. Cross-match: Phone+IP pairs with >1 user
phone_ip = (
    phones.merge(ips, on="user_id")
          .dropna(subset=["phone", "ip"])
          .drop_duplicates()
)

pair_counts = (
    phone_ip.groupby(["phone", "ip"])["user_id"]
            .nunique()
            .reset_index(name="count")
            .sort_values("count", ascending=False)
)

suspicious_pairs = pair_counts[pair_counts["count"] > 1]
print(f"Phone+IP pairs with >1 unique user: {len(suspicious_pairs)}")

# ---- 6. Top 5 suspicious overlaps with user lists
top5_keys = suspicious_pairs.head(5)[["phone", "ip"]]

top5_users = (
    phone_ip.merge(top5_keys, on=["phone", "ip"])
            .groupby(["phone", "ip"])["user_id"]
            .unique()
            .reset_index(name="people")
)

top5 = (
    suspicious_pairs.merge(top5_users, on=["phone", "ip"])
                    .sort_values(["count", "phone", "ip"], ascending=[False, True, True])
                    .head(5)
                    .reset_index(drop=True)
)

print("\nTop 5 suspicious phone+IP overlaps:")
print(top5)

# ---- 7. Save outputs
assets = Path("../assets")
assets.mkdir(parents=True, exist_ok=True)

csv_path = assets / "phase3_top5.csv"
top5.to_csv(csv_path, index=False)
print(f"\nSaved CSV: {csv_path}")

# ---- 8. Generate Markdown table (optional for README)
def to_md(df):
    out = ["| Phone | IP | Count | People |", "|---|---:|---:|---|"]
    for _, r in df.iterrows():
        people_str = ", ".join(map(str, r["people"]))
        out.append(f"| {r['phone']} | {r['ip']} | {int(r['count'])} | {people_str} |")
    return "\n".join(out)

md = to_md(top5)
md_path = assets / "phase3_top5.md"
md_path.write_text(md, encoding="utf-8")
print(f"Saved Markdown: {md_path}")
print("\nMarkdown table:\n")
print(md)

Phones shared by >1 user: 23
IPs shared by >1 user: 18
Phone+IP pairs with >1 unique user: 8

Top 5 suspicious phone+IP overlaps:
                    phone               ip  count  \
0      (873)330-8776x1949    137.167.55.87      5   
1       (878)958-5496x511  223.204.219.231      5   
2     +1-590-586-9987x961    211.154.39.34      5   
3        001-508-954-9123      28.209.7.32      5   
4  001-752-441-3430x26488   199.141.114.39      5   

                                              people  
0  [C998029122, C380101107, C978035840, C14307286...  
1  [C25552315, C1916496762, C1979501645, M5828861...  
2  [C807101143, C691065771, C1292484867, C5559908...  
3  [C1185590750, C473249, M702530586, C1308018080...  
4  [C1653380908, C153833710, C1002270002, C276022...  

Saved CSV: ..\assets\phase3_top5.csv
Saved Markdown: ..\assets\phase3_top5.md

Markdown table:

| Phone | IP | Count | People |
|---|---:|---:|---|
| (873)330-8776x1949 | 137.167.55.87 | 5 | C998029122, C380101107, C9780